In [1]:
import census
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format # avoid scientific notation

In [2]:
# Define outputs and inputs, read data
home = str(Path.home())
output_path = home+'/Documents/'
df = pd.read_csv(output_path + 'census_summ_2018.csv')

In [26]:
df = df.loc[df['county_y']==53]

In [27]:
# find the margin of error columns 
moe_cols = df.columns.str.contains('moe')
df.columns[moe_cols]

Index(['pop_18_moe', 'pop_nhislat_18_moe', 'white_18_moe', 'blk_18_moe',
       'aian_18_moe', 'asian_18_moe', 'nhopi_18_moe', 'sora_18_moe',
       'two_18_moe', 'twosora_18_moe', 'twoexc_18_moe', 'hislat_18_moe',
       'pop_12_moe', 'pop_nhislat_12_moe', 'white_12_moe', 'blk_12_moe',
       'aian_12_moe', 'asian_12_moe', 'nhopi_12_moe', 'sora_12_moe',
       'two_12_moe', 'twosora_12_moe', 'twoexc_12_moe', 'hislat_12_moe'],
      dtype='object')

In [28]:
# print all columns in the df
# for i in df.columns:
#     print("'"+i+"'"+",")

In [29]:
# Create a copy of the data frame with the data points that we need 
pop_calcs = df[['GEO_ID_y','state_y','county_y','tract_y','pop_12','pop_12_moe','pop_18','pop_18_moe',
                'pop_nhislat_12','pop_nhislat_12_moe','pop_nhislat_18','pop_nhislat_18_moe',
                'white_12','white_12_moe','white_18','white_18_moe',
                'blk_12','blk_12_moe','blk_18','blk_18_moe',
                'aian_12','aian_12_moe','aian_18','aian_18_moe',
                'asian_12','asian_12_moe','asian_18','asian_18_moe',
                'nhopi_12','nhopi_12_moe','nhopi_18','nhopi_18_moe',
                'hislat_12','hislat_12_moe','hislat_18','hislat_18_moe']].copy()

In [30]:
# first is enumerate then the column name; Create new columns based on titles 
for i,j in enumerate(pop_calcs.columns):
    if i<len(pop_calcs.columns):
        if str(j)+ "_moe"== pop_calcs.columns[i+1]:
            pop_calcs[str(j)+"moe_pct"] = pop_calcs[pop_calcs.columns[i+1]]/pop_calcs[pop_calcs.columns[i]]

In [31]:
# Remove inf values and replace with nan
pop_calcs.replace([np.inf, -np.inf], np.nan, inplace=True)

In [32]:
pop_calcs['poc_12'] = pop_calcs.loc[:,('pop_12')]- pop_calcs.loc[:,('white_12')]
pop_calcs['poc_18'] = pop_calcs.loc[:,'pop_18']-pop_calcs.loc[:,'white_18']

pop_calcs['poc_chg_1218'] = (pop_calcs.loc[:,'poc_18'] - pop_calcs.loc[:,'poc_12'])/ pop_calcs.loc[:,'poc_12']

In [37]:
pop_calcs.loc[pop_calcs['poc_chg_1218']<33].describe()

,state_y,county_y,tract_y,pop_12,pop_12_moe,pop_18,pop_18_moe,pop_nhislat_12,pop_nhislat_12_moe,pop_nhislat_18,pop_nhislat_18_moe,white_12,white_12_moe,white_18,white_18_moe,blk_12,blk_12_moe,blk_18,blk_18_moe,aian_12,aian_12_moe,aian_18,aian_18_moe,asian_12,asian_12_moe,asian_18,asian_18_moe,nhopi_12,nhopi_12_moe,nhopi_18,nhopi_18_moe,hislat_12,hislat_12_moe,hislat_18,hislat_18_moe,pop_12moe_pct,pop_18moe_pct,pop_nhislat_12moe_pct,pop_nhislat_18moe_pct,white_12moe_pct,white_18moe_pct,blk_12moe_pct,blk_18moe_pct,aian_12moe_pct,aian_18moe_pct,asian_12moe_pct,asian_18moe_pct,nhopi_12moe_pct,nhopi_18moe_pct,hislat_12moe_pct,hislat_18moe_pct,poc_12,poc_18,poc_chg_1218
count,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,160.00,162.00,140.00,133.00,165.00,169.00,93.00,109.00,170.00,171.00,171.00,171.00,171.00
mean,53.00,53.00,125613.12,4665.75,392.42,5014.19,404.02,4238.44,391.94,4485.47,406.44,3278.57,374.18,3381.44,377.92,307.25,152.77,322.61,150.83,46.22,49.08,54.07,58.08,272.35,133.15,306.32,134.12,59.54,62.60,69.04,70.54,427.31,221.75,528.71,217.73,0.09,0.08,0.10,0.10,0.12,0.12,0.80,0.71,1.22,1.26,0.67,0.60,1.26,1.22,0.62,0.50,1387.19,1632.75,0.29
std,0.00,0.00,214205.13,1432.41,146.28,1665.11,149.48,1320.33,126.08,1525.35,140.15,1192.57,118.98,1351.95,129.06,320.30,120.55,320.48,116.06,51.88,43.63,68.11,60.33,268.91,85.17,272.92,87.73,112.77,85.21,102.63,81.75,339.01,142.00,399.89,118.74,0.03,0.03,0.03,0.03,0.04,0.04,0.49,0.39,0.42,0.65,0.31,0.28,0.46,0.46,0.24,0.20,948.30,1024.27,0.45
min,53.00,53.00,60300.00,787.00,55.00,816.00,89.00,699.00,128.00,680.00,133.00,496.00,121.00,435.00,113.00,0.00,3.00,0.00,2.00,0.00,3.00,0.00,3.00,0.00,11.00,0.00,3.00,0.00,2.00,0.00,4.00,0.00,12.00,29.00,28.00,0.01,0.01,0.03,0.03,0.05,0.04,0.23,0.21,0.45,0.39,0.14,0.16,0.49,0.38,0.20,0.15,187.00,232.00,-0.45
25%,53.00,53.00,70257.00,3746.50,277.50,3959.50,302.50,3361.00,301.00,3485.00,308.50,2513.50,289.00,2473.00,293.50,46.50,52.50,66.00,60.00,8.50,17.00,2.50,17.00,96.50,70.00,111.50,68.50,0.00,12.00,0.00,17.00,183.00,112.50,274.00,133.00,0.07,0.07,0.08,0.08,0.09,0.10,0.46,0.39,0.94,0.93,0.48,0.42,0.93,0.91,0.47,0.34,631.00,830.50,0.00
50%,53.00,53.00,71705.00,4598.00,380.00,4933.00,391.00,4074.00,370.00,4301.00,389.00,3205.00,361.00,3354.00,373.00,199.00,135.00,204.00,119.00,29.00,36.00,28.00,33.00,201.00,117.00,214.00,115.00,8.00,17.00,18.00,28.00,357.00,191.00,429.00,195.00,0.08,0.08,0.09,0.09,0.12,0.11,0.64,0.59,1.15,1.10,0.59,0.54,1.19,1.20,0.61,0.48,1115.00,1370.00,0.20
75%,53.00,53.00,72904.00,5481.50,491.50,5947.00,492.50,5087.50,483.50,5531.00,473.50,4002.00,446.00,4185.00,453.00,480.00,227.00,515.50,219.50,62.00,62.50,81.50,80.50,358.00,183.00,389.00,176.00,70.00,84.00,109.00,113.00,607.50,300.50,662.00,280.00,0.11,0.10,0.12,0.11,0.15,0.14,1.04,0.93,1.49,1.45,0.79,0.73,1.47,1.54,0.72,0.63,1981.50,2198.50,0.47
max,53.00,53.00,940011.00,9258.00,813.00,11811.00,871.00,8067.00,731.00,10484.00,807.00,6165.00,741.00,8241.00,733.00,1884.00,620.00,1407.00,584.00,261.00,266.00,470.00,303.00,1709.00,478.00,1484.00,432.00,724.00,449.00,623.00,556.00,1913.00,704.00,2389.00,773.00,0.22,0.27,0.25,0.30,0.28,0.33,3.50,2.00,3.00,6.50,1.88,1.67,2.91,4.00,2.50,1.11,5963.00,6013.00,2.44


In [34]:
pop_calcs.loc[pop_calcs['poc_chg_1218']>=33]

,GEO_ID_y,state_y,county_y,tract_y,pop_12,pop_12_moe,pop_18,pop_18_moe,pop_nhislat_12,pop_nhislat_12_moe,pop_nhislat_18,pop_nhislat_18_moe,white_12,white_12_moe,white_18,white_18_moe,blk_12,blk_12_moe,blk_18,blk_18_moe,aian_12,aian_12_moe,aian_18,aian_18_moe,asian_12,asian_12_moe,asian_18,asian_18_moe,nhopi_12,nhopi_12_moe,nhopi_18,nhopi_18_moe,hislat_12,hislat_12_moe,hislat_18,hislat_18_moe,pop_12moe_pct,pop_18moe_pct,pop_nhislat_12moe_pct,pop_nhislat_18moe_pct,white_12moe_pct,white_18moe_pct,blk_12moe_pct,blk_18moe_pct,aian_12moe_pct,aian_18moe_pct,asian_12moe_pct,asian_18moe_pct,nhopi_12moe_pct,nhopi_18moe_pct,hislat_12moe_pct,hislat_18moe_pct,poc_12,poc_18,poc_chg_1218
398,1400000US53053060200,53,53,60200,684.00,122.00,2414.00,371.00,684.00,122.00,1388.00,222.00,639.00,125.00,873.00,147.00,7.00,12.00,155.00,120.00,1.00,3.00,8.00,12.00,20.00,23.00,289.00,131.00,16.00,27.00,18.00,41.00,0.00,12.00,1026.00,304.00,0.18,0.15,0.18,0.16,0.20,0.17,1.71,0.77,3.00,1.50,1.15,0.45,1.69,2.28,NaN,0.30,45.00,1541.00,33.24


# Analysis from the Data 

It seems like there was a 48% increase of people of color across all 172 census tracts in King County with a maximum of 3300% increase in POC in Census Tract: 53053060200 which seems to be the [Port of Tacoma](https://www.citivelocity.com/citybuilder/eppublic/cb/us/cities/15012/zones/53053060200) area of Tacoma Washington. I think that this brings up the mean way too much given that the median is only a 20% increase. When we remove the outlier, the mean is brought down to 29% increase on average with the median staying at 20%. We can use this change in POC to do a couple of things relating to finding relationships, but either way, we are at a good starting point for further analysis. Awaiting the other datasets to bring in. 